# Import all the necessary Libraries



In [0]:
import numpy as np
import json

## Load the training data

In [88]:
with open('train.json', 'rb') as f:
          j = json.load(f)
print(j.keys())
X = j['states']
Y = j['labels']

dict_keys(['states', 'labels'])


In [89]:
X_train = np.array(X)
y_train = np.array(Y).reshape(-1,1)
print(X_train.shape,y_train.shape)
#X_train

(750, 4) (750, 1)


### Extract the total number of classes available in labels

In [90]:
classes_data = set(Y) 
print(classes_data)

{'right', 'keep', 'left'}


### FInd the mean and variance of each variable based on each class and store them

* As we have three classes here L K R we have to store the mean and variance for the each variable respective to these classes.

* Variables 4 so total mean points 4 * 3 = 12
total variance points 4 * 3 =12


In [91]:
total_train_samples = X_train.shape[0]
total_variables = X_train.shape[1]
print(total_train_samples,total_variables)

750 4


In [0]:
temp_var_data = X_train[:,0].reshape(-1,1)
temp_var_data.shape
tt = temp_var_data[y_train=='left']

In [93]:
mean_v = []
variance_v = []

temp_var_data = np.array([])
for i in range(total_variables):
  m_v = []
  v_v = []
  for each in classes_data:
    temp_var_data = X_train[:,i].reshape(-1,1)
    sorted_data = temp_var_data[y_train==each]
    m_v.append(np.mean(sorted_data))
    v_v.append(np.var(sorted_data))
  mean_v.append(m_v)
  variance_v.append(v_v)

print(mean_v)
print(variance_v)




[[19.47718533601552, 20.324249331305595, 19.714133843021123], [2.934049463734621, 3.6802398635260003, 5.051805971302731], [9.947171857828826, 9.998538905035847, 9.91412576886327], [0.954021902134653, 0.005812036946454296, -0.9670872636504063]]
[[146.05850093405553, 130.78888470766984, 151.0414504886513], [5.346639057464283, 11.58599676646852, 5.570111210118603], [0.9064665992294042, 1.1419618439300834, 0.9805727278868248], [0.4184079876349627, 0.02826634149781208, 0.4399430283399336]]


In [94]:
mean_v[1][0]

2.934049463734621

In [0]:
def gpb(x,mu,var):
  numer = np.exp(-0.5*((x-mu)**2)/var)
  denom = 1/np.sqrt(2*(np.pi)*var)  
  return (numer*denom)

In [96]:
gpb(0.99999,1,0.5)

0.5641895834913373

In [0]:
pb_x =np.array([1/3,1/3,1/3,1/3]).reshape(-1,1)
def pred_class(pred_data,prob_x = pb_x,classes_data=classes_data):
  prob_classes = np.dot(pred_data.transpose(),prob_x)
  total_prob = np.sum(prob_classes)
  prob_classes = prob_classes/total_prob
  idx = np.argmax(prob_classes)

  return list(classes_data)[idx]

In [119]:
predict_data = np.ones((4,3))
print(predict_data)
print(np.matmul(predict_data.transpose(),np.array([0.5,0.5,0.5,0.5])))


[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
[2. 2. 2.]


In [0]:

def predict(x_test,mean_v = mean_v,variance_v =variance_v,classes_data=classes_data):
  predict_data = np.zeros((4,3))
  for i in range(len(x_test)):
      for j in range(len(classes_data)):
        x = x_test[i]
        mu = mean_v[i][j]
        var = variance_v[i][j]
        prob_c = gpb(x,mu,var)
        predict_data[i,j] = prob_c
  class_found = pred_class(predict_data)
  return class_found


In [0]:
x_test = [34.768072958280662, 0.83291381243218088, 8.2066386346429869, -0.99896114245770529]

In [127]:
predict_data = predict(x_test)
print(predict_data)

left


## Load the test data

In [0]:
with open('test.json', 'rb') as f:
          j = json.load(f)
X = j['states']
Y = j['labels']

In [136]:
wrong_pred = 0
success = 0
data_len = len(X)
for i in range(data_len):
  pred = predict(X[i])
  if(pred==Y[i]):
    success += 1 
  else:
    wrong_pred +=1

accuracy = (data_len-wrong_pred)*100/data_len
print('the total number of missed predictions are:', wrong_pred)
print('accuracy of the classifier is:', accuracy)

the total number of missed predictions are: 39
accuracy of the classifier is: 84.4
